In [1]:
import os
import json
from speach import elan
import string
import pandas as pd
import warnings
import re

In [2]:
def parsefile(file):
    punct = string.punctuation
    table = pd.read_csv(file, sep='\t')
    speakers = {i.split('@')[-1] for i in table.columns}
    glossing = []
    for speaker in speakers:
        table_A = table[[f'tx@{speaker}', f'mb@{speaker}', f'ge@{speaker}']]
        table_A.dropna(inplace=True)
        table_A = table_A.groupby(f'tx@{speaker}').aggregate({f'mb@{speaker}': list, f'ge@{speaker}': list})
        for row in table_A.itertuples():
            morph = 0
            for word in row[0].split():
                wordform = word.lower().strip(punct)
                glosses = []
                root = False
                if morph >= len(row[2]):
                    continue
                while morph < len(row[2]) and not(root == True and row[2][morph][0] not in ('-', '=')):
                    if row[2][morph][-1] in ('-', '='):
                        glosses.append(row[2][morph])
                    if row[2][morph][-1] not in ('-', '=') and row[2][morph][0] not in ('-', '='):
                        root = True
                        glosses.append(row[2][morph])
                    if row[2][morph][0] in ('-', '='):
                        glosses.append(row[2][morph])
                    morph += 1
                glossing.append((wordform, set(glosses)))
    return glossing

In [3]:
warnings.simplefilter('ignore')
glossing = []
for file in os.listdir('ELAN'):
    if file.split('.')[-1] == 'txt':
        glossing.extend(parsefile(f'ELAN/{file}'))        

In [4]:
def convarg(text):
    if text[:2].isnumeric():
        cl = 2
    else:
        cl = 1
    
    new = text[:cl]
    
    if text[cl].islower():
        new += text[cl:cl+2] + '.'
    
    if text[-2] == 'O':
        new += 'obj'
    elif text[-2] == 'S':
        new += 'sbj'
    
    return new

In [5]:
def is_Ruuli(word):
    for char in word:
        if char not in "bdjgptckvzfslrmnwyŋiueoa'":
            return False
    return True

In [6]:
with open('gram_tags.json', 'r', encoding='utf-8') as tagset:
    gram_tags = json.load(tagset)
gram_tags = set(gram_tags)

In [7]:
standard = []

In [8]:
for wordform in glossing:
    form = wordform[0].strip(string.punctuation+'”“…').replace("’", "'")
    if not is_Ruuli(form):
        continue
    glosses = wordform[1]
    new_glosses = []
    for gloss in glosses:
        
        
        if len(gloss.split('~')) > 1:
            gloss = gloss.split('~')[0]
        
        for part in re.split('\.|:', gloss):
            if part.lower().strip('~-=') == '':
                continue
            if part.lower().strip('~-=') in gram_tags:
                new_glosses.append(part.lower().strip('~-='))
            elif part.lower().strip('~-=') in ('aux', 'aiux'):
                new_glosses.append('cop')
            elif part.lower().strip('~-=') == 'interj':
                new_glosses.extend('intj')
            elif part.strip('~-=') == 'POSS':
                new_glosses.append('assoc')
            elif part.strip('~-=')[-4:] == 'POSS':
                new_glosses.append('assoc')
                new_glosses.append(part.replace('POSS', '').replace('.', '').lower().strip('~-='))
            elif part[0].isnumeric() and part[-2:] in ('O-', 'S-'):
                new_glosses.extend(convarg(part).split('.'))
            elif part[0].isnumeric() and part[-1] in ('O', 'S'):
                new_glosses.extend(part[:-1].lower())
                
    entry = (form, new_glosses)
    if entry not in standard and len(new_glosses) > 0:
        standard.append(entry)

In [9]:
with open('standard.json', 'w', encoding='utf-8') as f:
    json.dump(standard, f, ensure_ascii=False, indent='\t')